In [1]:
import requests


In [8]:
import json
import os 
import datetime
import hashlib
from flask import Flask, jsonify, request

### Need request for the decentralized network
## what makes a blockchain a crypto (transactions)

### Add Transacitons 
#### Make a consensus network to make sure ench node has the same chain - >
## All the blocks are up to date up to the last

In [5]:
from uuid import uuid4
## uuid for each node in network
from urllib.parse import urlparse

In [10]:
class Cardano:
    def __init__(self):
        ### Seperate list of transacitons before added to the block 
        ### Soone as a miner mines a block all the transactions are added to the block
        ### List of transacitons will be emptied
        self.chain = []
        self.transactions = []
        self.addCoin(proof = 1, prevHash = "0")
        ### node address will be ip from computers
        self.nodes = set()
        
    def addCoin(self, proof, prevHash):
        coin = {
            "index": len(self.chain),
            "timestamp": str(datetime.datetime.now()),
            "proof": proof,
            "prevHash": prevHash,
            "transactions": self.transactions
        }
        ## must empty List of transacitons
        self.transactions = []
        self.chain.append(coin)
        return coin
        
    def getLastCoin(self):
        return self.chain[-1]
    
    def hashCoin(self, coin):
        encodedCoin = json.dumps(coin, sort_keys=True).encode()
        return hashlib.sha256(encodedCoin).hexdigest()
    
    def proofOfWork(self, prevProof):
        new_proof = 1
        solved = False 
        
        while solved is False:
            hashWork = hashlib.sha256(str(new_proof ** 2 - prevProof ** 2).encode()).hexdigest()
            if hashWork[:4] == "0000":
                solved = True
            else:
                new_proof += 1
        return new_proof
    
    def isChainValid(self, chain):
        previousCoin = chain[0]
        current_index = 1 
        while current_index < len(chain):
            current_coin = chain[current_index]
            currentCoinPrevHash = current_coin["prevHash"]
            if currentCoinPrevHash != hashCoin(previousCoin):
                return False
            previousProof = previousCoin["proof"]
            currentProof = current_coin["proof"]
            hashWork = hashlib.sha256(str(currentProof ** 2 - previousProof ** 2).encode()).hexdigest()
            if hashWork[:4] != "0000":
                return False
            previousCoin = current_coin
            current_index += 1
        return True
    
       ## Create a new method to create format for transactions and append to the transaction list
    def addTransaction(self, sender, receiver, amount):
        self.transactions.append({
            "sender": sender, 
            "receiver": receiver,
            "amount": amount
        })
        ##return the index of the block that the transactions will go into 
        previousBlock = self.getLastCoin() 
        return previousBlock["index"] + 1
    
    def addNode(self, ipAddress):
        ## prase the address of the node 
        parsedURl = urlparse(ipAddress)
        self.nodes.add(parsedURl.netloc)
    
    def shouldReplaceChain(self):
        network = self.nodes
        maxLength = len(self.chain)
        longestChain = None
        
        for node in network:
            response = requests.get(f"http://{node}/chain")
            if response.status_code == 200:
                length = response.json()["length"]
                chain = respose.json()["chain"]
                if length > maxLength and self.isChainValid(chain):
                    maxLength = length
                    longestChain = chain
            if longestChain: ## not none
                self.chain = longestChain
                return True
            return False


In [45]:
cardano = Cardano()

In [46]:
app = Flask(__name__)

In [47]:
@app.route("/mine", methods = ["GET"])
def mine():
    previousCoin = cardano.getLastCoin()
    prevProof = previousCoin["proof"]
    proof = cardano.proofOfWork(prevProof)
    prevHash = cardano.hashCoin(previousCoin)
    coin = cardano.addCoin(proof, prevHash)
    response = {
        "index": coin["index"],
        "timestamp": coin["timestamp"],
        "proof": coin["proof"],
        "prevHash": coin["prevHash"],
        "previousCoin": previousCoin
    }
    return jsonify(response), 200

In [48]:
@app.route("/chain", methods = ["GET"])
def chain():
    response = {
        "chain": cardano.chain,
        "length": len(cardano.chain)
    }
    return jsonify(response), 200

In [ ]:
app.run(host="0.0.0.0", port=5000)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://0.0.0.0:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [29/Dec/2021 00:29:10] "GET /mine HTTP/1.1" 200 -
127.0.0.1 - - [29/Dec/2021 00:29:11] "GET /mine HTTP/1.1" 200 -
127.0.0.1 - - [29/Dec/2021 00:29:13] "GET /mine HTTP/1.1" 200 -
127.0.0.1 - - [29/Dec/2021 00:29:14] "GET /mine HTTP/1.1" 200 -
127.0.0.1 - - [29/Dec/2021 00:29:18] "GET /chain HTTP/1.1" 200 -
